### Reading data from silver layer

In [0]:
df_silver = spark.sql ("select * from parquet.`abfss://silver@thadiyalcardatalake.dfs.core.windows.net/carsales`")

###reading all the dims

In [0]:
df_dealer = spark.sql("select * from carproject.gold.dim_dealer")
df_branch =spark.sql ("select * from carproject.gold.dim_branch")
df_model = spark.sql("select * from carproject.gold.dim_model")
df_date = spark.sql("select * from carproject.gold.dim_date")

In [0]:
df_fact = df_silver.join(df_branch, df_silver.Branch_ID == df_branch.Branch_ID, "left")\
    .join(df_dealer, df_silver.Dealer_ID == df_dealer.Dealer_ID, "left")\
    .join(df_model, df_silver.Model_ID == df_model.Model_ID, "left")\
    .join(df_date, df_silver.Date_ID == df_date.Date_ID, "left")\
        .select(df_silver.Revenue, df_silver.Units_Sold, df_silver.RevPerUnit, df_branch.dim_branch_key, df_dealer.dim_dealer_key, df_model.dim_model_key, df_date.dim_date_key)


In [0]:
df_fact.limit(10).display()

Revenue,Units_Sold,RevPerUnit,dim_branch_key,dim_dealer_key,dim_model_key,dim_date_key
13363978,2,6681989.0,418,6,155,825
17376468,3,5792156.0,1557,197,252,825
9664767,3,3221589.0,1058,104,199,752
5525304,3,1841768.0,789,95,183,752
12971088,3,4323696.0,497,231,106,882
7321228,1,7321228.0,1804,41,41,988
11379294,2,5689647.0,734,177,107,988
11611234,2,5805617.0,1211,182,110,1043
19979446,2,9989723.0,116,204,185,1043
14181510,3,4727170.0,116,160,238,826


###Writing Fact table

In [0]:
from delta.tables import DeltaTable

In [0]:
if spark.catalog.tableExists("carproject.gold.factsales"):
    deltatbl = DeltaTable.forName(spark, "carproject.gold.factsales")
    deltatbl.alias('trg').merge(df_fact.alias('src'), "trg.dim_branch_key = src.dim_branch_key and trg.dim_dealer_key = src.dim_dealer_key and trg.dim_model_key = src.dim_model_key and trg.dim_date_key = src.dim_date_key")\
        .whenMatchedUpdateAll()\
        .whenNotMatchedInsertAll()\
        .execute()
else:
    df_fact.write.format('delta')\
        .mode('overwrite')\
        .option("path","abfss://gold@thadiyalcardatalake.dfs.core.windows.net/factsales")\
        .saveAsTable("carproject.gold.factsales")

In [0]:
%sql
select * from carproject.gold.factsales limit 10

Revenue,Units_Sold,RevPerUnit,dim_branch_key,dim_dealer_key,dim_model_key,dim_date_key
13363978,2,6681989.0,418,6,155,825
17376468,3,5792156.0,1557,197,252,825
9664767,3,3221589.0,1058,104,199,752
5525304,3,1841768.0,789,95,183,752
12971088,3,4323696.0,497,231,106,882
7321228,1,7321228.0,1804,41,41,988
11379294,2,5689647.0,734,177,107,988
11611234,2,5805617.0,1211,182,110,1043
19979446,2,9989723.0,116,204,185,1043
14181510,3,4727170.0,116,160,238,826
